# Заготовка для работы Частотный анализ звуков

импортируем библиотеки

In [ ]:
%pylab inline
from ipywidgets import interact
from IPython.display import Audio, display

# библиотеки для работы с сигналами
from scipy.signal import hilbert, periodogram, savgol_filter
from scipy.io import wavfile
import warnings
warnings.filterwarnings("ignore", category=wavfile.WavFileWarning)


## 1. Загрузка сигнала из файла

> *опишите источник данных. Какова была идея при записи? С какими трудностями пришлось столкнуться?  Переконвертируйте в формат wav, например, с помощью https://audio.online-convert.com/convert-to-wav или https://online-audio-converter.com/ru/*

In [ ]:
u='mono.wav' #имя файла. чтобы указать только имя (как здесь), поместите его в папку с блокнотом
fs,y = wavfile.read(u)   #читаем файл с помощью библиотеки wavfile
y.shape  #форма массива со значениями прочитанного файла

> *опишите переменные, которые были созданы при загрузке данных.
сделайте вывод о типе записи: моно или стерео.
(чтобы посмотреть значение переменной - создайте новую ячейку, вставьте имя переменной и выполните)*

In [ ]:
tmax = y.shape[0]/fs  # возьмите размер первого измерения (индекс 0) и поделите на ...
tmax

> прокомментируйте полученное число.

Звук записан в у.е., показывающих степень сжатия и разрежения воздуха. Традиционно смотрят развертку сигнала во времени. При этом по оси абсцисс откладывают время, а по оси ординат - амплитуду колебаний.

> _опишите график временной развертки сигнала: какие участки выделяются визуально? какие их длительность и амплитуда?
 сопоставьте выделяемые визуально участки со звуками, слышимыми при прослушивании._

In [ ]:
# если два канала, оставляем только первый
if y.ndim > 1:
    y = y[:,0]

t = arange(len(y))/fs  #создайте ряд временных отметок в сек.
plot(t, y);  # постройте график
xlabel('...'); # задайте подписи по осям
ylabel('...');

Хотя звук открылся в виде массива обычных чисел, мы можем послушать эти числа (и любые другие), задав скорость звучания (параметр `rate`). Можно замедлить или убыстрить проигрывание, просто изменяя этот параметр.

In [ ]:
Audio(y, rate=fs, autoplay=True)

> _Сделайте вывод о возможности выделения отдельных звуков на основании изменения амплитуды колебаний.
Поставьте задачу следующего этапа: сколько и каких фрагментов Вы должны получить?_


## 2. Выделение отдельных звуков

> _как называется способ для обнаружения начала и концов амплитудных всплесков, который использует разложение Гильберта в следующем фрагменте кода?_

In [ ]:
k_sglazh = 1555  # задайте нечетное число - ширину окна сглаживания
og = savgol_filter(np.abs(hilbert(y)), k_sglazh, 2)

fig, ax = subplots(1,1, figsize=(12,3)) # задайте удлиненную форму рисунка
plot(t,y, label='сигнал');  # нарисуйте исходный сигнал, если закомментировать эту строку - будет легче выбрать порог
plot(t,og,'r', label='огибающая'); # сверху наложите огибающую
xlabel('...'); # задайте подписи по осям
ylabel('...');
# расскомментируйте следующую строчку, чтобы выделить фрагмент
#xlim(8,10)  #укажите начало и конец фрагмента в сек

> _опишите процедуру подбора ширины окна сглаживания. Какое значение позволит сгладить  огибающую кривую так, чтобы для каждого выделяемый звук приходился один подъем и один спуск огибающей ?_

> _обоснуйте, почему для демонстрации качества сглаживания Вы выбрали именно этот фрагмент записи?_

Зададим порог вполовину от максимума, или меньше, и выделим все участки сигнала превышающие порог.

> _Индексы начала звука найдем, как места где индикатор превышения порога растет, а индексы окончания - где снижается. Попробуйте объяснить логику рассчета переменных iiab и iiad (индексы), если смысл функций astype(int) - перевод в целое число, diff - производная (разница между последующими и предыдущими значениями), where - (англ. где) позиции, где условие в скобках истинно_ 

> _попробуйте включать и выключать выделенные блоки кода с помощью знака комментирования #  комбинацией клавиш **Ctrl-/** в английской раскладке_

In [ ]:
porog=4000
bb = og>porog # True (1) - где звук, False (0) - где тишина

iiab = where( diff( bb.astype(int) ) == 1)[0]+1
iiad = where( diff( bb.astype(int) ) == -1)[0]+1


# ниже код для удаления дребезга, откомментируйте для просмотра

# okno_udaleniya_drebezga = 20000 
# isi = iiad[1:]-iiab[:-1]  # расчитайте Интервал между звуками
# ii_drebezg = where(isi < okno_udaleniya_drebezga)[0]
# iiab = delete(iiab ,ii_drebezg+1)
# iiad = delete(iiad ,ii_drebezg)


# ниже код для вспомогательного графика, откомментируйте для просмотра

# amax = max(y)  # найдем максимальную амплитуду для рисования вертикальных линий
# figsize(12,3);
# plot(og, label='огибающая');
# vlines(iiab, 0, amax, color='red', label='начало')
# vlines(iiad, 0, amax, color='green', label='конец')
# legend();
# #xlim(190000, 220000)  # выделение фрагмента


#число точек смены состояний должно совпасть!
dd = (iiad-iiab)/fs * 1000 # рассчитайте длительность в мс., уберите 1000 для сек
iiab, iiad, dd


> _опишите операции по подбору порога и окна удаления дребезга.С какими трудностями пришлось столкнуться?_

> _сделайте вывод о соответствии количества найденных индексов начала и оконачания звуков задаче, поставленной в начале этого этапа_

> _сделайте вывод о длительности отдельных звуков - какие звуки самые короткие и длинные (это пригодится при выборе единой длительности_

> _в случае сомнения подтвердите свой вывод рисунками с характеристиками найденных фрагментов c помощью следующего кода:_

```py
imz = diff(iiab+dd)/fs #рассчитайте интервал между центрами звуков
a = [max(_x) for i,_x in enumerate(split(y, sort(hstack([iiab,iiad])))) if i%2]
figsize(12,3);
subplot(131)
bar(arange(len(dd)), dd); title('Длительность звуков, мс');
subplot(132)
bar(arange(len(imz)), imz); title('Интервал между звуками, с');
subplot(133)
bar(arange(len(a)), a); title('Громкость, у.е.');
```

## 3. Спектральный анализ 

При расчете частотных спектров фрагменты сигнала должны иметь некоторую минимальную длительность. Поскольку частота оцифровки звука высока, то данных обычно достаточно даже в очень коротких фрагментах. Параметр `NFFT` (количество компонент при [разложении Фурье](https://ru.wikipedia.org/wiki/Быстрое_преобразование_Фурье)) задает подробность выделения отдельных частотных составляющих. Если данных не хватает, этот параметр можно снизить вдвое или вчетверо.

Построим спектр для каждого из выделенных фрагментов.

In [ ]:
figsize(10,5)
for iab,iad in zip(iiab,iiad):
    y1 = y[iab:iad]
    P, ff = csd(y1, y1, NFFT=500, Fs=fs);
    
legend(arange(len(iiab))+1);
xlim(0,4000); # ограничьте частотный диапазон для наглядного отображения

> _Охарактеризуйте распределение спектральной плотности. На каких частотах есть различия между звуками? какие звуки похожи по спектральному составу?_

Для сравнения спектрограмм отдельных звуков между собой лучше взять одинаковые отрезки сигнала, отступив от начальных отметок назад и вперед. 

> _Можете попробовать выбрать длину отрезка, опираясь на секунды. В этом случае значение в секундах надо умножить на частоту сигнала `fs`._

На каждой итерации цикла также будем складывать полученные значения в массив `X`.

In [ ]:
X = []

# подготовьте подрисунки для всех фрагментов
fig, axx = subplots(1,len(iiab), sharex=True, sharey=True, figsize=(14,3)) #, subplot_kw=dict(xticks=[0.05,0.1]))

for i,iab in enumerate(iiab):  #для каждого индекса в цикле
    y1 = y[iab-1000 : iab+9000] # выделите фрагмент равной длины с отступами взад-вперед 
    sca(axx[i]) #подготовьте соответствующий подрисунок
    Pxx, freqs, bins, im = specgram(y1, Fs=fs, NFFT=500, noverlap=0, cmap='viridis');
    title(i+1)
    X.append(Pxx.flatten()) # добавьте значения в массив Х

X = np.asarray(X)
X.shape

> _Кроме того, что значения спектрограмм отобразились на рисунках, они также сохранились в массив X.  Опишите размеры массива (выводятся на экран последней строчкой кода). Почему они такие?_

>_Меняются ли размеры массива, если изменять отступы для выделения фрагментов?
Если Ваши звуки больше по длительности, чем полсекунды, то обязательно увеличьте отступы в строке `y1 = y[iab-1000 : iab+9000]`, чтобы на спектрограммах была видна динамика развития спектральной плотности, связанной с каждым звуком (появление и исчезновение светлого пятна на темном фоне).
Обоснуйте свой выбор._


> _Параметр `cmap=` задает цветовую гамму для отображения значений. Числа изменяются только по величине, поэтому сопоставление со цветом условно. Какая из цветовых карт Вам кажется самой адекватной? Попробуйте 'gray', 'jet', 'autumn', 'winter'_


> _**Для любознательных** Как меняется зерно рисунка, если выключить параметр `noverlap` (noverlap=0) и изменять параметры `NFFT` (уменьшить до NFFT=50)?_

> _Опишите спектрограммы - в каких временных (Ox) и частотных (Oy) координатах значения наиболее информативны? После какой частоты различия между звуками не просматриваются?
Как соотносятся пятна на спектрограммах со звучанием выделенных Вами звуков._

## Сохранение результатов для дальнейшего использования

Сохраним массив значений спектральной плотности для всех выделенных звуков в текстовом формате.


In [ ]:
savetxt('spectral_data.txt', X)

> _проверьте появление файла в рабочей папке. Обратите внимание на размер и дату изменения._

Для переноса нескольких переменных в другой блокнот удобно сохранить их в одном сжатом файле. 

In [ ]:
savez('для рисунка', og=og, fs=fs, iiab=iiab, iiad=iiad)

> _Если вы хотите сохранить данные для фрагментов из нескольких разных файлов, то повторите анализ в другой копии этого блокнота. Чтобы выходные данные из ваших блокнотов были совместимыми - проследите, чтобы длительность выделяемых фрагментов была одинаковой._

> # _Критерии оценки_

> Критерии оценивания за расчетно-графическую работу (баллов):
- работа сдана в правильном формате +30
- работа сдана в форме реферата +10
- название не по шаблону -1
- не убраны инструкции и комментарии в конце строчек кода -5
- не везде комментарии, нет выводов -5
- источник данных не изменен -10
- источник данных выбран из предложенных -8
- источник данных взят из интернета -3
- источник данных не соответствует требованиям -3
- некорректное суждение при ответе на вопрос -2 за каждое 